In [ ]:
import tensorflow as tf
from tensorflow import keras as K
import numpy as np
import argparse
from progressbar import ProgressBar
import os

In [ ]:
class Network:
    def __init__(self, session, dict_weight, dropout=0.3, lstm_units=1024, dense_units=60):
        self.sess = session
        K.backend.set_session(self.sess)
        #defining layers
        dict_shape = dict_weight.shape
        self.emb = K.layers.Embedding(dict_shape[0], dict_shape[1], weights=[dict_weight], trainable=False, name='embedding')
        self.drop = K.layers.Dropout(rate=dropout, seed=91, name='dropout')
        self.lstm = K.layers.LSTM(lstm_units, stateful=False, return_sequences=False, name='lstm')
        
        self.dense = K.layers.Dense(dense_units, activation='relu', name='dense')
        self.p = K.layers.Dense(1, activation='sigmoid', name='p')
        #defining optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)

    def __call__(self, batch, perturbation=None):
        
        batch1 = batch[:,0:400]
        batch2 = batch[:,400:800]
        batch3 = batch[:,800:1200]
        
        embedding1 = self.emb(batch1)
        embedding2 = self.emb(batch2)
        embedding3 = self.emb(batch3)
        
        drop1 = self.drop(embedding1)
        drop2 = self.drop(embedding2)
        drop3 = self.drop(embedding3)
            
        if (perturbation is not None):
            drop1 += perturbation
            drop2 += perturbation
            drop3 += perturbation
        
        lstm1 = self.lstm(drop1)
        lstm2 = self.lstm(drop2)
        lstm3 = self.lstm(drop3)
        
        lstm = tf.concat([lstm1, lstm2, lstm3], axis=1)
        dense = self.dense(lstm)
        
        return self.p(dense), embedding1, embedding2, embedding3,
    
    def get_minibatch(self, x, y, ul, batch_shape=(16, 1200)):
        x = K.preprocessing.sequence.pad_sequences(x, maxlen=batch_shape[1])
        permutations = np.random.permutation( len(y) )
        ul_permutations = None
        len_ratio = None
        if (ul is not None):
            ul = K.preprocessing.sequence.pad_sequences(ul, maxlen=batch_shape[1])
            ul_permutations = np.random.permutation(len(ul))
            len_ratio = len(ul)/len(y)
        for s in range(0, len(y), batch_shape[0]):
            perm = permutations[s:s+batch_shape[0]]
            minibatch = {'x': x[perm], 'y': y[perm]}
            if (ul is not None):
                ul_perm = ul_permutations[int(np.floor(len_ratio*s)):int(np.floor(len_ratio*(s+batch_shape[0])))]
                minibatch.update( {'ul': np.concatenate((ul[ul_perm], x[perm]), axis=0)} )             
            yield minibatch
    
    def get_loss(self, batch, labels):
        pred, emb1, emb2, emb3 = self(batch)
        loss = K.losses.binary_crossentropy(labels, pred)
        return tf.reduce_mean( loss ), emb1, emb2, emb3
    
    def get_adv_loss(self, batch, labels, loss, emb, p_mult):
        gradient = tf.gradients(loss, emb, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
        p_adv = p_mult * tf.nn.l2_normalize(tf.stop_gradient(gradient), dim=1)
        adv_loss = K.losses.binary_crossentropy(labels, self(batch, p_adv)[0])
        return tf.reduce_mean( adv_loss )
    
    def get_v_adv_loss(self, ul_batch, p_mult, power_iterations=1):
        bernoulli = tf.distributions.Bernoulli
        prob, emb = self(ul_batch)
        prob = tf.clip_by_value(tf.stop_gradient(prob), 1e-7, 1.-1e-7)
        prob_dist = bernoulli(probs=prob)
        #generate virtual adversarial perturbation
        d = tf.random_uniform(shape=tf.shape(emb), dtype=tf.float32)
        for _ in range( power_iterations ):
            d = (0.05) * tf.nn.l2_normalize(d, dim=1)
            p_prob = tf.clip_by_value(self(ul_batch, d)[0], 1e-7, 1.-1e-7)
            kl = tf.distributions.kl_divergence(prob_dist, bernoulli(probs=p_prob), allow_nan_stats=False)
            gradient = tf.gradients(kl, [d], aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
            d = tf.stop_gradient(gradient)
        d = p_mult * d
        #virtual adversarial loss
        p_prob = tf.clip_by_value(self(ul_batch, d)[0], 1e-7, 1.-1e-7)
        v_adv_loss = tf.distributions.kl_divergence(prob_dist, bernoulli(probs=p_prob), allow_nan_stats=False)
        return tf.reduce_mean( v_adv_loss )
    
    def validate(self, xval, yval, batch_shape=(64, 1200), log_path=None):
        print( 'Validation...' )
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='batch')

        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        accuracies = list()
        bar = ProgressBar(max_value=np.floor(len(ytest)/batch_shape[0]).astype('i'))
        minibatch = enumerate(self.get_minibatch(xval, yval, None, batch_shape=batch_shape))
        for i, test_batch in minibatch:
            fd = {batch: val_batch['x'], labels: val_batch['y'], K.backend.learning_phase(): 0} #test mode
            accuracies.append( self.sess.run(accuracy, feed_dict=fd) )
            bar.update(i)
        
        log_msg = "\nValidation Average accuracy is {:.3f} -- batch shape {}"
        print( log_msg.format(np.asarray(accuracies).mean(), batch_shape) )
        log = None
        if(log_path is not None):
            log = open(log_path, 'a')
            log.write(log_msg+'\n')
            log.close()
    
    def train(self, dataset, batch_shape=(64, 1200), epochs=10, loss_type='none', p_mult=0.02, save=None, log_path=None):
        print( 'Training...' )
        xtrain = np.load( "{}nltk_xtrain.npy".format(dataset) )
        ytrain = np.load( "{}nltk_ytrain.npy".format(dataset) )
        
        # get a validation set of one batch
        xval = list()
        yval = list()
        for i in range(int(len(ytrain)*0.025)):
            xval.append(xtrain[0])
            xval.append(xtrain[-1])
            yval.append(ytrain[0])
            yval.append(ytrain[-1])
            np.delete(xtrain, 0, 0)
            np.delete(xtrain, -1, 0)
            np.delete(ytrain, 0, 0)
            np.delete(ytrain, -1, 0)
        xval = np.asarray(xval)
        yval = np.asarray(yval)
        yval = np.reshape(yval, newshape=(yval.shape[0],1))
            
        ytrain = np.reshape(ytrain, newshape=(ytrain.shape[0], 1))
        ultrain = np.load( "{}nltk_ultrain.npy".format(dataset) ) if (loss_type == 'v_adv') else None 
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='batch')
        ul_batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='ul_batch')
        
        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        loss, emb1, emb2, emb3 = self.get_loss(batch, labels)
        if (loss_type == 'adv'):
            loss += self.get_adv_loss(batch, labels, loss, emb, p_mult)
        elif (loss_type == 'v_adv'):
            loss += self.get_v_adv_loss(ul_batch, p_mult)

        opt = self.optimizer.minimize( loss )
        #initializing parameters
        self.sess.run( [var.initializer for var in tf.global_variables() if not('embedding' in var.name)] )
                       
        _losses = list()
        _accuracies = list()
        log = None
                         
        list_ratio = (len(ultrain)/len(ytrain)) if (ultrain is not None) else None
        for epoch in range(epochs):
            losses = list()
            accuracies = list()
            
            bar = ProgressBar(max_value=np.floor(len(ytrain)/batch_shape[0]).astype('i'))
            minibatch = enumerate(self.get_minibatch(xtrain, ytrain, ultrain, batch_shape=batch_shape))
            for i, train_batch in minibatch:
                fd = {batch: train_batch['x'], labels: train_batch['y'], K.backend.learning_phase(): 1} #training mode
                if (loss_type == 'v_adv'):
                    fd.update( {ul_batch: train_batch['ul']} )
                    
                _, acc_val, loss_val = self.sess.run([opt, accuracy, loss], feed_dict=fd)
                
                accuracies.append( acc_val )
                losses.append( loss_val )
                bar.update(i)
            
            _losses.append(losses)
            _accuracies.append(accuracies)
            
            log_msg = "\nEpoch {} of {} -- average accuracy is {:.3f} -- average loss is {:.3f}"
            print( log_msg.format(epoch+1, epochs, np.asarray(accuracies).mean(), np.asarray(losses).mean()) )
            if(log_path is not None):
                log = open(log_path, 'a')
                log.write(log_msg+'\n')
                log.close()
        
            #validation
            self.validate(xval, yval, batch_shape=batch_shape, log_path=log_path)
                     
        #plotting value
        #plt.plot([l for loss in _losses for l in loss], color='magenta', linestyle='dashed', marker='s', linewidth=1)
        plt.plot([np.asarray(l).mean() for l in _losses], color='red', linestyle='solid', marker='o', linewidth=2)
        #plt.plot([a for acc in _accuracies for a in acc], color='cyan', linestyle='dashed', marker='s', linewidth=1)
        plt.plot([np.asarray(a).mean() for a in _accuracies], color='blue', linestyle='solid', marker='o', linewidth=2)
        if loss_type == 'adv':
            plt.savefig('./img/adv/fancy_train_e{}_m{}_l{}.png'.format(epochs, batch_shape[0], batch_shape[1]))
        elif loss_type == 'v_adv':
            plt.savefig('./img/v_adv/fancy_train_e{}_m{}_l{}.png'.format(epochs, batch_shape[0], batch_shape[1]))
        else:
            plt.savefig('./img/base/fancy_train_e{}_m{}_l{}.png'.format(epochs, batch_shape[0], batch_shape[1]))
        
    def test(self, dataset, batch_shape=(64, 1200), log_path=None):
        print( 'Test...' )
        xtest = np.load( "{}nltk_xtest.npy".format(dataset) )
        ytest = np.load( "{}nltk_ytest.npy".format(dataset) )
        ytest = np.reshape(ytest, newshape=(ytest.shape[0], 1))
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='batch')

        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        
        accuracies = list()
        bar = ProgressBar(max_value=np.floor(len(ytest)/batch_shape[0]).astype('i'))
        minibatch = enumerate(self.get_minibatch(xtest, ytest, None, batch_shape=batch_shape))
        for i, test_batch in minibatch:
            fd = {batch: test_batch['x'], labels: test_batch['y'], K.backend.learning_phase(): 0} #test mode
            accuracies.append( self.sess.run(accuracy, feed_dict=fd) )
            bar.update(i)
        
        log_msg = "\nTest Average accuracy is {:.3f} -- batch shape {}"
        print( log_msg.format(np.asarray(accuracies).mean(), batch_shape) )
        
        log = None
        if(log_path is not None):
            log = open(log_path, 'a')
            log.write(log_msg+'\n')
            log.close()

In [ ]:
def main(data, n_epochs, n_ex, ex_len, lt, pm):
    os.environ["CUDA_VISIBLE_DEVICES"] = '0'
    config = tf.ConfigProto(log_device_placement=True)
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)

    embedding_weights = np.load( "{}nltk_embedding_matrix.npy".format(data) )

    # log text file
    if(lt=='none'):
        net_tp ='baseline'
    else:
        net_tp = lt
    log_path ='./res/fancy_'+str(net_tp)+'_bs_'+str(n_ex)+'_ep_'+str(n_epochs)+'_sl_'+str(ex_len)+'.txt'
    log = open(log_path, 'w')
    log.write('Fancy Network with {} loss, {} epochs, {} batch size, {} maximum string length \n'.format(net_tp, n_epochs, n_ex, ex_len))
    log.close()
    
    #print(log_path)
                                  
    net = Network(session, embedding_weights)
    net.train(data, batch_shape=(n_ex, ex_len), epochs=n_epochs, loss_type=lt, p_mult=pm, log_path=log_path)
    net.test(data, batch_shape=(n_ex, ex_len), log_path=log_path)
    
    
    K.backend.clear_session()


In [ ]:
if (__name__ == '__main__'):
    parser = argparse.ArgumentParser()
    parser.add_argument('path', help='path of the folder that contains the data (train, test, emb. matrix)', type=str)
    parser.add_argument('epochs', help='number of training epochs', type=int)
    parser.add_argument('--nex', help='Number of EXamples per batch', default=64, type=int)
    parser.add_argument('--exlen', help='LENght of each EXample', default=1200, type=int)
    parser.add_argument('--loss', help='define the loss type', choices=['none', 'adv', 'v_adv'], default='none', type=str)
    parser.add_argument('--p_mult', help='Perturbation MULTiplier (used with adv and v_adv)', default=0.02, type=float)
    args = parser.parse_args()
    
    main(args.path, args.epochs, args.nex, args.exlen, args.loss, args.p_mult)

In [ ]:
# main(data='../dataset/imdb/', n_epochs=5, n_ex=64, ex_len=1200, lt='none', pm=0.1)